leetcode 176第二高的薪水
如果不存在第二高的薪水，那么查询应返回 null

select nvl(max(salary), NULL) SecondHighestSalary
from employee
where salary < (select max(distinct salary) from Employee

如果表达式1为空值，NVL返回值为表达式2的值

select emp_no, salary 
from (
select emp_no, salary, row_number() over(order by salary desc) t
from salaries
where to_date = '9999-01-01')
where t = 2

leetcode 177 第N高的薪水

CREATE FUNCTION getNthHighestSalary(N IN NUMBER) RETURN NUMBER IS
result NUMBER;
BEGIN

    select--内层
        nvl(Salary, NULL) INTO result
    from (--外层
        select 
            distinct Salary, dense_rank() over(order by Salary desc) rn 
        from Employee
    ) t where rn=N;

    RETURN result;
END;

dense_rank()为分区或结果集中的每一行分配排名，而排名值没有间隙 1 2 2 3 4 4 5

leetcode 184 部门工资最高的员工

select c.name as department,
b.name as employee,
a.salary
from (select departmentid,max(salary) as salary from employee group by departmentid) a
join employee b on a.departmentid=b.departmentid and a.salary=b.salary
join department c on a.departmentid=c.id

leetcode 185 部门工资前三高的所有员工

select Department,Employee,Salary
from(
    select d.name Department,e.name Employee, Salary,
    dense_rank() over(partition by DepartmentId order by Salary desc) rk
    from Employee e inner join Department d
    on e.DepartmentId = d.Id
    )t
where rk<=3

SQL 21 查找在职员工自入职以来的薪水涨幅情况

select b.emp_no,(b.salary-a.salary) as growth
from
(select e.emp_no,s.salary
from employees e left join salaries  s on e.emp_no=s.emp_no
and e.hire_date=s.from_date)a -- 入职工资表
inner join
(select e.emp_no,s.salary
from employees e left join salaries  s on e.emp_no=s.emp_no
where s.to_date='9999-01-01')b -- 现在工资表
on a.emp_no=b.emp_no
order by growth

SQL 23 对所有员工的薪水按照salary降序进行1-N排序

SELECT emp_no, salary,
dense_rank () over (ORDER BY salary DESC) AS `rank`
FROM salaries WHERE to_date = '9999-01-01' 

SQL 24 获取所有非manager员工当前的薪水情况
select de.dept_no,a.emp_no,s.salary
from
(select emp_no
from employees
where emp_no not in (select emp_no
from dept_manager)
) a
inner join dept_emp de on a.emp_no=de.emp_no
inner join salaries s on a.emp_no=s.emp_no
where s.to_date='9999-01-01'

SQL25 获取员工其当前的薪水比其manager当前薪水还高的相关信息
SELECT a.emp_no, b.manager_no, a.emp_salary, b.manager_salary
FROM
(SELECT de.dept_no, de.emp_no, s.salary AS emp_salary
FROM dept_emp de, salaries s
WHERE de.emp_no=s.emp_no
AND s.to_date='9999-01-01') a,
(SELECT dm.dept_no, dm.emp_no AS manager_no, s.salary AS manager_salary
FROM dept_manager  dm, salaries s
WHERE dm.emp_no=s.emp_no
AND s.to_date='9999-01-01') b
WHERE a.dept_no=b.dept_no
AND a.emp_salary>b.manager_salary;

SQL26 汇总各个部门当前员工的title类型的分配数目
select distinct d.dept_no, d.dept_name, t.title, count(t.emp_no)over(partition by t.title , de.dept_no)
from departments as d join dept_emp as de on d.dept_no = de.dept_no
                      join titles as t on t.emp_no = de.emp_no
where de.to_date = '9999-01-01'
and t.to_date = '9999-01-01'
order by d.dept_no;

SQL28 查找描述信息中包含robot的电影对应的分类名称以及电影数目
select c.name,
       count(fc.film_id)
from film f,category c,film_category fc
where f.description like '%robot%'
and f.film_id=fc.film_id
and fc.category_id=c.category_id
and c.category_id in (select category_id
                      from film_category
                      group by category_id
                       having count(film_id)>=5)

建表
CREATE TABLE actor(
actor_id smallint(5) primary key,
first_name varchar(45) not null,
last_name varchar(45) not null,
last_update date not null);

常规创建
create table if not exists 目标表
复制表格
create 目标表 like 来源表
将table1的部分拿来创建table2
create table if not exists actor_name
(
first_name varchar(45) not null,
last_name varchar(45) not null
)
select first_name,last_name
from actor

新建一列
alter table actor add create_date datetime not null default '2020-10-01 00:00:00'

插入
INSERT INTO actor(actor_id,
                  first_name,
                  last_name,
                  last_update)
VALUES(1,'PENELOPE','GUINESS','2006-02-15 12:34:33'),
      (2,'NICK','WAHLBERG','2006-02-15 12:34:33');
      
insert into actor
values(3,'ED','CHASE','2006-02-15 12:34:33');

删除
DELETE FROM titles_test
WHERE id NOT IN(
    SELECT * FROM(
    SELECT MIN(id)
    FROM titles_test
    GROUP BY emp_no)a); 
    
UPDATE 表名称 SET 列名称 = 新值 WHERE 列名称 = 某值 update titles_test set to_date = null , from_date = '2001-01-01' where to_date = '9999-01-01'


每个人的最近登录时间
请你统计一下新登录用户的次日成功的留存率
第一天登录的用户
select user_id,min(date) from login group by user_id

select 
round(count(distinct user_id)*1.0/(select count(distinct user_id) from login) ,3)
from login
where (user_id,date)
in (select user_id,date(min(date),'+1 day') from login group by user_id);

每天登录的新用户数
select distinct date
        ,sum(case when (user_id,date) in 
    (select user_id,min(date)from login group by user_id)
    then 1 else 0 end)
from login
group by date
order by date;

SQL 70 每个日期新用户的次日留存率
select date
        ,ifnull(round((sum(case when (user_id,date)in
            (select user_id,date_add(date,interval -1 day) 
             from login)  and (user_id,date)in (select user_id,min(date)from login group by user_id)
            then 1 else 0 end))/
        (sum(case when (user_id,date)in
            (select user_id,min(date)from login group by user_id)
            then 1 else 0 end)),3),0)as p
from login
group by date
order by date;